In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from itertools import combinations
import lxml
import traceback
import re
from bs4 import BeautifulSoup as bs
from thefuzz import fuzz

In [ ]:
authors_df = pd.read_csv('econlit_scopus_matching_out/JPE_author_abstract_funding.csv')

In [ ]:
aer_df = pd.read_csv('econlit_scopus_matching_out/aer_econlit.csv')
eca_df = pd.read_csv('econlit_scopus_matching_out/eca_econlit.csv')
jpe_df = pd.read_csv('econlit_scopus_matching_out/jpe_econlit.csv')
qje_df = pd.read_csv('econlit_scopus_matching_out/qje_econlit.csv')
res_df = pd.read_csv('econlit_scopus_matching_out/res_econlit.csv')
rje_df = pd.read_csv('econlit_scopus_matching_out/rje_econlit.csv')

aer_df['publication'] = 'aer'
eca_df['publication'] = 'eca'
jpe_df['publication'] = 'jpe'
qje_df['publication'] = 'qje'
res_df['publication'] = 'res'
rje_df['publication'] = 'rje'


econlit_df = pd.concat([aer_df, eca_df, jpe_df, qje_df, res_df, rje_df], ignore_index=True)

In [ ]:
# abstracts_df = authors_df[['doi', 'sc_title', 'sc_abstract_text']]
abstracts_df = econlit_df[['title', 'date', 'year', 'publication', 'abstract', 'jel_code', 'L_code', 'K_code', 'D4_code', 'O3_code', 'G34_code']]

abstracts_df['anti_trust_indicator'] = 0
abstracts_df['market_power_indicator'] = 0
abstracts_df['L4_code'] = 0
abstracts_df['K21_code'] = 0


anti_trust_pattern = r'anti trust|anti-trust|antitrust'
market_power_pattern = r'market power|market-power'

In [ ]:
for row in range(0, len(abstracts_df)):
    # sc_abstract_text 

    abstract = abstracts_df.loc[row, 'abstract']
    publication = abstracts_df.loc[row, 'publication']
    if type(abstracts_df.loc[row, 'abstract']) !=str:
        continue
    if re.search(anti_trust_pattern, abstract, flags=re.I):
        # print("Anti-trust: {}".format(row))
        abstracts_df.loc[row, 'anti_trust_indicator']=1
        print('ANTI-TRUST--{} in abstract: {}'.format(publication, row))
    if re.search(market_power_pattern, abstract, flags=re.I):
        # print("Market power: {}".format(row))
        abstracts_df.loc[row, 'market_power_indicator']=1
        print('MARKET-POWER--{} in abstract: {}'.format(publication, row))
    # if re.search(r'merger', abstract, flags=re.I):
    #     print('MERGER--{} in abstract: {}'.format(publication, row))
    # if re.search(r'cartel', abstract, flags=re.I):
    #     print('CARTEL--{} in abstract: {}'.format(publication, row))
    # if re.search(r'monopol', abstract, flags=re.I):
    #     print('MONOPOL--{} in abstract: {}'.format(publication, row))
    # if re.search(r'deadweight loss|deadweight|dead-weight|dead weight', abstract, flags=re.I):
    #     print('DEADWEIGHT LOSS--{} in abstract: {}'.format(publication, row))
    # if re.search(r'anti compet|anti-compet|anticompet', abstract, flags=re.I):
    #     print('ANTI-COMPET--{} in abstract: {}'.format(publication, row))

    if type(abstracts_df.loc[row, 'jel_code']) != str:
        continue
    if re.search(r'L4', abstracts_df.loc[row, 'jel_code'], flags=re.I):
        abstracts_df.loc[row, 'L4_code'] = 1

    if re.search(r'K21', abstracts_df.loc[row, 'jel_code'], flags=re.I):
        abstracts_df.loc[row, 'K21_code'] = 1

    


        
# print('ANTI-TRUST COUNT: {}'.format(len(abstracts_df[abstracts_df.anti_trust_indicator==1])))
# print('MARKET POWER COUNT: {}'.format(len(abstracts_df[abstracts_df.market_power_indicator==1])))
print('L4-AER CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='aer')])))
print('K21-AER CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='aer')])))
print('L4-ECA CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='eca')])))
print('K21-ECA CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='eca')])))
print('L4-jpe CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='jpe')])))
print('K21-jpe CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='jpe')])))
print('L4-qje CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='qje')])))
print('K21-qje CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='qje')])))
print('L4-res CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='res')])))
print('K21-res CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='res')])))
print('L4-rje CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.L4_code==1) & (abstracts_df.publication=='rje')])))
print('K21-rje CODE COUNT: {}'.format(len(abstracts_df[(abstracts_df.K21_code==1) & (abstracts_df.publication=='rje')])))


In [232]:
theory_codes_pattern = r'C7|D11|D5|D21|D85|D86'

L_papers_df = abstracts_df[abstracts_df.L_code==1]
L_papers_df.reset_index(inplace=True)
L_papers_df['contains_theory'] = 0

for i in range(0, 10):
    generated_key = 'L' + str(i)+ '_code'
    L_papers_df[generated_key]=0

for j in [0,1,2,3,4,9]:
    generated_key = 'L4' + str(j) + '_code'
    L_papers_df[generated_key] = 0

for row in range(0,len(L_papers_df)):
    if re.search(theory_codes_pattern, L_papers_df.loc[row, 'jel_code'], flags=re.I):
        L_papers_df.loc[row, 'contains_theory'] =1

    for i in range(0, 10):
        generated_pattern = r'L'+ str(i)
        generated_column = 'L' + str(i) + '_code'
        if re.search(generated_pattern, L_papers_df.loc[row, 'jel_code'], flags=re.I):
            L_papers_df.loc[row, generated_column] = 1

    for j in [0,1,2,3,4,9]:
        generated_pattern = r'L4'+str(j)
        generated_column = 'L4' + str(j) + '_code'
        if re.search(generated_pattern, L_papers_df.loc[row, 'jel_code'], flags=re.I):
            L_papers_df.loc[row, generated_column] = 1
            print(generated_pattern)
            print(abstracts_df.loc[row, 'jel_code'])


print("TOTAL IO: {}".format(len(L_papers_df)))
print("TOTAL ANTITRUST: {}".format(len(L_papers_df[L_papers_df.L4_code ==1])))
print("IO and at least one theory : {}".format(len(L_papers_df[L_papers_df.contains_theory==1])))
print("ANTITRUST AND THEORY: {}".format(len(L_papers_df[(L_papers_df.contains_theory==1) & (L_papers_df.L4_code == 1)]))) 
## Plot these counts and shares by year

L_papers_df.to_csv('../adhoc_material/Reports/04-21/L_papers_indicators.csv', index=False, encoding='utf-8')
# L_papers_df

C:\Users\Joshualevy\AppData\Local\Temp\ipykernel_16836\2222349206.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L_papers_df['contains_theory'] = 0
C:\Users\Joshualevy\AppData\Local\Temp\ipykernel_16836\2222349206.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L_papers_df[generated_key]=0
C:\Users\Joshualevy\AppData\Local\Temp\ipykernel_16836\2222349206.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

L41
['D31', 'G51', 'J15', 'J24', 'N31', 'N32']
L42
['G22', 'G52', 'G53', 'Q54']
L40
['I25', 'I26', 'J24', 'N34', 'R23']
L44
['C90', 'D81']
L40
['D82', 'D83', 'G14']
L41
['D24', 'D43', 'G34', 'L13', 'L25', 'L94']
L41
['E43', 'G12', 'G15']
L41
['D22', 'H25', 'H26', 'L25', 'O14', 'O17']
L42
['F23', 'H25', 'H32', 'L25']
L42
[nan]
L42
['E21', 'E31', 'E43', 'E52']
L41
['D24', 'F13', 'L25', 'L60', 'O14', 'P31', 'P33']
L43
['D24', 'F23', 'L13', 'L71', 'Q35']
L42
['D14', 'D15', 'D91', 'G41']
L41
['C92', 'D83', 'D91', 'J16']
L42
['D83', 'D84', nan, 'E23', 'E32']
L40
['I12', 'Q51', 'Q53', 'Q58']
L40
['I23', 'I26']
L44
['D82', 'D83', 'G14', 'G18']
L40
[nan]
L42
['C21', 'J13', 'J68']
L40
['D13', 'D14', 'I12', 'J13', 'J14']
L42
['H51', 'I11', 'I14', 'I18', 'J12', 'J13', 'O15']
L41
['E23', 'E24', 'E32', 'J11', 'J14', 'O33']
L41
['A14', 'J00']
L41
['A14', 'O10']
L41
['A14']
L41
['A11']
L42
['Y90']
L42
['D72', 'E32', 'G01', 'N10']
L41
['D11', 'D21', 'D83', 'L13', 'L25', 'L71', 'L81']
L42
['H51', 'I13',

OSError: [Errno 22] Invalid argument: '../adhoc_material/Reports/04-21/L_papers_indicators.csv'

In [ ]:
year_pub_count = pd.DataFrame()
year_pub_count['count'] = abstracts_df.groupby(by=['year', 'publication']).size()
indicator_count_df = abstracts_df.groupby(by=['year', 'publication']).agg({
    'L_code': lambda x: x.sum(),
    'K_code': lambda x: x.sum(),
    'L4_code': lambda x: x.sum(),
    'K21_code': lambda x: x.sum(),
    'D4_code': lambda x: x.sum(),
    'O3_code': lambda x: x.sum(),
    'G34_code': lambda x: x.sum(),
    'anti_trust_indicator': lambda x: x.sum(),
    'market_power_indicator': lambda x: x.sum(),
})
indicator_count_df.reset_index(inplace=True)
indicator_count_df = pd.merge(indicator_count_df, year_pub_count, how='left', on=['year', 'publication'])
indicator_count_df.to_csv('indicator_count_df.csv', index=False, encoding='utf-8')
# indicator_count_df

In [ ]:
# aer_authors_df = pd.read_csv('econlit_scopus_matching_out/AER_author_abstract_funding.csv')
eca_authors_df = pd.read_csv('econlit_scopus_matching_out/ECA_author_abstract_funding.csv')
jpe_authors_df = pd.read_csv('econlit_scopus_matching_out/JPE_author_abstract_funding.csv')
qje_authors_df = pd.read_csv('econlit_scopus_matching_out/QJE_author_abstract_funding.csv')
res_authors_df = pd.read_csv('econlit_scopus_matching_out/RES_author_abstract_funding.csv')
rje_authors_df = pd.read_csv('econlit_scopus_matching_out/RJE_author_abstract_funding.csv')

authors_df = pd.concat([eca_authors_df, jpe_authors_df, qje_authors_df, res_authors_df, rje_authors_df], ignore_index=False)
authors_df = authors_df[['doi', 'sc_title', 'sc_pub_name', 'sc_author_given_name', 'sc_author_last_name', 'sc_author_id']]
authors_df.reset_index(inplace=True)

In [ ]:
for row in range(0, len(authors_df)):
    pub_name = authors_df.loc[row, 'sc_pub_name']
    if type(pub_name) != str:
        continue

    if pub_name == 'Econometrica : journal of the Econometric Society':
        authors_df.loc[row, 'sc_pub_name'] = 'Econometrica'
    elif pub_name == 'The Rand journal of economics':
        authors_df.loc[row, 'sc_pub_name'] = 'RAND Journal of Economics'

In [ ]:
authors_df.drop_duplicates(subset=['doi', 'sc_pub_name', 'sc_author_id'], inplace=True)

In [ ]:
# test_df = authors_df.head(100)
coauthors_df = authors_df.head(1000).groupby(by=['doi']).apply(lambda df: unique_author_pairs(df))
coauthors_df.reset_index(inplace=True)
coauthors_df[['author', 'coauthor']] = pd.DataFrame(coauthors_df['pairs'].tolist(), index=coauthors_df.index)
# coauthors_df

In [ ]:
def unique_author_pairs(sub_df):
    author_ids_list = sub_df['sc_author_id'].unique().tolist()
    doi = sub_df['doi'].unique()[0]
    
    if len(author_ids_list) ==1:
        unique_author_pairs = pd.Series(pair for pair in combinations([author_ids_list[0], np.nan], 2))
    else:
        unique_author_pairs = pd.Series(pair for pair in combinations(author_ids_list,2))
    # print(doi)
    # print(author_ids_list)
    # print(unique_author_pairs)
    
    # print('-----------------------------')
    return pd.DataFrame({
        # 'doi' : doi,
        'pairs' : unique_author_pairs
    })

In [239]:
G = nx.from_pandas_edgelist(coauthors_df, 'author', 'coauthor')
# G = nx.from_pandas_adjacency(coauthors_df[['author', 'coauthor']])

G.remove_node(np.nan)
nan_edges = [edge for edge in G.edges if np.nan in edge]
print(nan_edges)
# G.remove_edges_from(nan_edges)
# for tup in G.edges:
#     if np.nan in tup:
#         source = tup[0]
#         target = tup[1]
#         print(tup)
#         G.remove_edge(source, target)
from matplotlib.pyplot import figure
figure(figsize=(20,15))
# nx.draw_shell(G, node_size=2)
# nx.draw_random(G, node_size=5)
# nx.draw_networkx_edges(G)
pos = nx.kamada_kawai_layout(G)
nx.draw(G, pos=pos)  # Draw the original graph

[]


FileNotFoundError: Could not find module 'C:\Users\Joshualevy\Anaconda3\envs\ioCapture\lib\site-packages\scipy\.libs\libbanded5x.JL7Z4SENYRPJEE7ARLMB5S7URXMJTATU.gfortran-win_amd64.dll' (or one of its dependencies). Try using the full path with constructor syntax.

<Figure size 1440x1080 with 0 Axes>